In [1]:
!mkdir -p data
!mkdir -p data_input

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df_enc = pd.read_csv('data_win/enc.csv').set_index('hosp_id')
df_pop = pd.read_csv('data_win/windows.csv').set_index('hosp_id')
df_win = pd.read_csv('data_win/windows_map.csv')

In [4]:
df_win[['ID']].to_csv('data_input/pop.csv', index=False)

## Filter vars

### Meds

In [5]:
df_meds = pd.read_pickle('data_win/meds-2020_0701-win.p')

In [6]:
len(df_meds)

2681426

In [7]:
# Filter by year
df_meds = df_meds[df_meds['hosp_id'].isin(df_enc.index)]

In [ ]:
df_meds

In [9]:
keep = np.loadtxt('/data/covid/process_data/NEW-meds-2015-2019/data/meds-keep.txt')

In [10]:
df_out = df_meds[df_meds['medicationtermid'].isin(keep)]

In [ ]:
df_out

In [12]:
df_out.to_pickle('data/meds-filtered.p')

### Labs

In [13]:
df = pd.read_pickle('data_win/labs-2020_0701-win.p')

In [ ]:
df

In [15]:
keep = np.loadtxt('/data/covid/process_data/FIDDLE_2018-2019/data/labs-keep.txt')

In [16]:
df_out = df[df['result_termid'].isin(keep)]

In [ ]:
df_out

In [18]:
df_out.to_pickle('data/labs-filtered.p')

## Format IDs

### Meds

In [19]:
df_meds = pd.read_pickle('data/meds-filtered.p')

In [20]:
df_meds['medicationtermid'].nunique()

958

In [21]:
len(df_meds)

2504094

In [22]:
df_meds_small = df_meds[df_meds['hosp_id'].isin(df_enc.index)]

In [23]:
len(df_meds_small)

2504094

In [24]:
df_meds_small = df_meds_small[(df_meds_small['t'] >= 0)]

In [25]:
len(df_meds_small)

2504094

In [26]:
df_meds_ID = df_meds.set_index(['hosp_id', 'window_id']).join(df_win.set_index(['hosp_id', 'window_id'])[['ID']]).reset_index()

In [27]:
df_meds_ID.to_pickle('data_input/meds_ID.p')

In [28]:
df = df_meds_ID[['ID', 'window_t', 'medicationtermid', 'dose', 'route']] \
.rename(columns={'window_t': 't', 'medicationtermid': 'variable_name'})

In [29]:
df_attr = df.melt(id_vars=['ID', 't', 'variable_name'], var_name='attribute', value_name='variable_value')
df_attr['variable_name'] = df_attr['variable_name'].astype(str).str.cat(df_attr['attribute'], sep='_')
df_attr = df_attr.drop(columns=['attribute'])
df_attr = df_attr.drop_duplicates(subset=['ID', 't', 'variable_name'], keep='last') # Need to improve handling of duplicates

In [ ]:
df_attr

In [31]:
df_attr.to_pickle('data_input/meds.p')

### Labs

In [32]:
df_labs = pd.read_pickle('data/labs-filtered.p')

In [ ]:
df_labs

In [34]:
df_labs['result_termid'].nunique()

1010

In [35]:
len(df_labs)

4961256

In [36]:
df_labs['hilonormal_flag'] = df_labs['hilonormal_flag'].fillna('N')

In [37]:
df_labs_small = df_labs[df_labs['hosp_id'].isin(df_enc.index)]

In [38]:
len(df_labs_small)

4961256

In [39]:
df_labs_small = df_labs_small[(df_labs_small['t'] >= 0)]

In [40]:
len(df_labs_small)

4961256

In [41]:
df_labs_ID = df_labs.set_index(['hosp_id', 'window_id']).join(df_win.set_index(['hosp_id', 'window_id'])[['ID']]).reset_index()

In [42]:
df_labs_ID.to_pickle('data_input/labs_ID.p')

In [43]:
df = df_labs_ID[['ID', 'window_t', 'result_termid', 'value', 'hilonormal_flag']] \
.rename(columns={'window_t': 't', 'result_termid': 'variable_name'})

In [44]:
df_attr = df.melt(id_vars=['ID', 't', 'variable_name'], var_name='attribute', value_name='variable_value')
df_attr['variable_name'] = df_attr['variable_name'].astype(str).str.cat(df_attr['attribute'], sep='_')
df_attr = df_attr.drop(columns=['attribute'])
df_attr = df_attr.drop_duplicates(subset=['ID', 't', 'variable_name'], keep='last') # Need to improve handling of duplicates

In [ ]:
df_attr

In [46]:
df_attr.to_pickle('data_input/labs.p')